Taxi Database
http://www.andresmh.com/nyctaxitrips/

In [26]:
# read in an convert the trip data to h5
def convert_trip():
    tr = pd.read_csv('../data/final.csv',
                      chunksize=2000000,
                      names="medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_date,pickup_time,dropoff_date,dropoff_time,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude".split(','),
                      dtype={'store_and_fwd_flag' : object})

    def parse_dates(df, name):

        date = df.pop("%s_date" % name)
        time = df.pop("%s_time" % name)
        df[name] = pd.to_datetime(date + ' ' + time,infer_datetime_format=True)
        return df

    categories = {}
    def parse_cats(df, name):

        field = df.pop(name)
        cats = categories.get(name)
        df[name] = pd.Categorical(field, categories=cats)
        if cats is None:
            categories[name] = df[name].cat.categories
        return df

    store = pd.HDFStore('../data/final.h5',mode='w')
    for i, df in enumerate(tr):
        
        # convert dates
        for name in ['pickup','dropoff']:
            df = parse_dates(df, name)

        # convert objects to cats
        for name in ['medallion','hack_license','vendor_id','store_and_fwd_flag']:
            df = parse_cats(df, name)

        store.append('df',df,format='table',data_columns=True,index=False)
    store.close()
    
def convert_fare():
    import glob
    store = pd.HDFStore('../data/fare.h5',mode='w')
    for file in glob.glob('../data/trip_fare_*.csv'):
        df = pd.read_csv(file)
        df.columns = df.columns.to_series().str.strip()
        
        categories = {}
        def parse_cats(df, name):

            field = df.pop(name)
            cats = categories.get(name)
            df[name] = pd.Categorical(field, categories=cats)
            if cats is None:
                categories[name] = df[name].cat.categories
            return df

        # convert dates
        df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
        
        # convert objects to cats
        for name in ['medallion','hack_license','vendor_id','payment_type']:
            df = parse_cats(df, name)

        store.append('df',df,format='table',data_columns=True,index=False)
    store.close()

In [27]:
%timeit -n 1 -r 1 convert_trip()

1 loops, best of 1: 7min 40s per loop


In [3]:
%timeit -n 1 -r 1 convert_fare()

1 loops, best of 1: 48.7 s per loop


In [20]:
# define the grouper
def groupby(store, grouper, field, gfunc=None):
    """ return sum, counts for the grouped field """
    nrows = store.get_storer('df').nrows

    def chunker():
        chunksize = 5000000
        for i, chunk in enumerate(range(nrows/chunksize)):

            start = chunk*chunksize
            stop = min((chunk+1)*chunksize,nrows)
            yield start, stop
        
    sums = []
    counts = []
    for start, stop in chunker():
        df = store.select('df',start=start,stop=stop,columns=[grouper,field])
        df.columns = ['grp','data']
        
        if gfunc is not None:
            df['grp'] = gfunc(df['grp'])
        g = df.groupby('grp')['data']
        sums.append(g.sum())
        counts.append(g.count())

    sums = pd.concat(sums)
    counts = pd.concat(counts)
    sums = sums.groupby(level='grp').sum()
    counts = counts.groupby(level='grp').sum()

    return sums, counts

In [24]:
with pd.get_store('../data/final.h5') as store:
    print store.get_storer('df').nrows
    
!ls -ltr ../data/final.*

98835718
-rw-r--r--  1 jreback  staff  16467220329 Nov 16 20:00 ../data/final.csv
-rw-r--r--  2 jreback  staff   9292830544 Nov 17 17:45 ../data/final.h5


In [22]:
def groupby_medallion_counts():
    with pd.get_store('../data/final.h5') as store:
        sums, counts  = groupby(store, 'medallion', 'passenger_count')
        #print sums/counts
%timeit -n 1 -r 1 groupby_medallion_counts()

1 loops, best of 1: 22.9 s per loop


In [4]:
with pd.get_store('../data/fare.h5') as store:
    print store.get_storer('df').nrows
    
!ls -ltr ../data/*fare*

14776615
-rwxrwxrwx@ 1 jreback  staff  1681610043 Aug 25  2013 ../data/trip_fare_1.csv
-rw-r--r--  1 jreback  staff  1036057072 Nov 17 19:43 ../data/fare.h5


In [23]:
# chunking
def groupby_avg_fares():
    with pd.get_store('../data/fare.h5') as store:
        sums, counts = groupby(store, 'pickup_datetime', 'fare_amount', gfunc=lambda x: x.dt.day)
        #print sums/counts
%timeit -n 1 -r 1 groupby_avg_fares()

1 loops, best of 1: 6.58 s per loop


In [1]:
# in memory
x = pd.read_hdf('../data/fare.h5','df',columns=['pickup_datetime','fare_amount']).set_index('pickup_datetime')
%timeit -n 1 -r 1 x.resample('D',how='mean')

AssertionError: cannot interpret dtype of [category] in [name->medallion,cname->medallion,dtype->category,kind->None,shape->None]